In [ ]:
import anthropic
import dotenv
import base64

In [4]:

anthropic_api_key = dotenv.get_key(dotenv_path=".env", key_to_get="ANTHROPIC_API_KEY")
if anthropic_api_key is None:
    Exception("APi key not available")


In [5]:
MODEL_NAME='claude-3-5-sonnet-20241022'

In [6]:
client = anthropic.Anthropic(api_key=anthropic_api_key)

In [7]:
def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as image_file:
        binary_data = image_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode('utf-8')
        return base64_string

In [15]:
def get_response(image_path: str):
    message_list = [
        {
            "role": 'user',
            "content": [
                {"type": "image", "source": {"type": "base64", "media_type": "image/png", "data": get_base64_encoded_image(image_path=image_path)}},
            ]
        }
    ]

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=message_list,
        system='You are a data scientist and you are working on a project to extract information from a diagram in json format. Which is compatiable with knowledge graph databases. Consider their shape and translate it\'s purpose(process, decision, etc.) as it is important for the data extraction. only give the json format of the diagram.',
    )
    return response

In [16]:
response = get_response(image_path='./images/image.png')

In [17]:
print(response.content[0].text)

{
  "nodes": [
    {
      "id": "start",
      "type": "terminator",
      "label": "Start"
    },
    {
      "id": "initialize",
      "type": "process",
      "label": "Initialize"
    },
    {
      "id": "decision",
      "type": "decision",
      "label": "Decision"
    },
    {
      "id": "process",
      "type": "process",
      "label": "Process"
    },
    {
      "id": "delay",
      "type": "delay",
      "label": "Delay"
    },
    {
      "id": "print_result",
      "type": "process",
      "label": "Print result"
    },
    {
      "id": "end",
      "type": "terminator",
      "label": "End"
    }
  ],
  "edges": [
    {
      "from": "start",
      "to": "initialize",
      "type": "normal"
    },
    {
      "from": "initialize",
      "to": "decision",
      "type": "normal"
    },
    {
      "from": "decision",
      "to": "process",
      "label": "No",
      "type": "dashed"
    },
    {
      "from": "process",
      "to": "delay",
      "type": "dashed"
    }